In [45]:
from pydantic import BaseModel
from openai import OpenAI
import json

with open('secret.json', 'r') as file:
    secrets = json.load(file)


client = OpenAI(
  api_key = secrets["XVisure"]["KEY"]
)


In [46]:
import pdfplumber

pdf_path = "data/Fascicolo.pdf"
with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())


In [120]:
from datetime import datetime
from pydantic import BaseModel, Field, field_validator, ValidationError, field_serializer
from typing import Any, List, Annotated
import re

def serialize_date(date: datetime, format=None) -> str:
    if isinstance(date, str):
        return date
    
    if format is None:
        return date.strftime("%d/%m/%Y")
    else:
        return date.strftime(format)

class Indirizzo(BaseModel):
    tipo_via: str = Field(title="Tipo di via", description="Tipologia del nome della via es. Via, Piazza, Contrà, Contrada Corso, Viale, Passerella, Strada")
    via: str = Field(title="Nome della via", description="Denominazione del personaggio, della cosa o del luogo a cui è intitolata la via es. Roma in Via Roma, del Popolo in Piazza del Popolo, Cantarane in Via Cantarane, Giuseppe Mazzini in via Mazzini (o G. Mazzini o Giuseppe Mazzini)  San Giovanni in Contrada San Giovanni, Italia in Corso Italia, Europa in Viale Europa,  dei Mulini in Passerella dei Mulini, Pini in Strada dei Pini")
    cap: str = Field(title="Codice di avviamento postale", description="Codice postale o Codice di avviamento postale, numero identificativo del luogo ai fini postali")
    comune: str = Field(title="Comune", description="Nome del comune, nome della più bassa entità territoriale amministrativa")
    provincia: str = Field(title="Provincia o Contea", description="Nome della provincia o contea, nome dell'entità territoriale amministrativa di livello immediatamente superiore che include il comune")
    regione: str = Field(title="Regione o Stato Federale", description="Nome della regione o stato federale, nome dell'entità territoriale amministrativa di livello immediatamente superiore che include la provincia o contea")
    paese: str = Field(title="Paese", description="Nome del paese, nome dell'entità territoriale amministrativa riconosciuta come nazione, paese, stato")


class Sede(Indirizzo):
    tipo_sede: str = Field(title="Tipo di sede", description="Tipologia della sede es. Sede legale, Sede amministrativa, Sede operativa, Sede di riferimento")

class Attivita(BaseModel):
    stato_attivita: bool = Field(title="Stato attività", description="Indica se l'attività è attiva o inattiva, se attiva è True, se inattiva è False")
    prevalente_boolean: bool = Field(title="Attività prevalente", description="Indica se l'attività è prevalente")
    data_inizio_attivita: str = Field(title="Data inizio attività", description="Data di inizio dell'attività")
    descrizione_attivita_prevalente: str = Field(title="Descrizione attività prevalente", description="Descrizione dell'attività prevalente")
    ateco: str = Field(title="Codice ATECO", description="Codice ATECO, codice identificativo dell'attività economica")
    nace: str = Field(title="Codice NACE", description="Codice NACE, codice identificativo dell'attività economica")
    import_export_boolean: bool = Field(title="Import/Export", description="Indica se l'attività è di import/export")



class ValAttivita(Attivita):
        @field_validator("stato_attivita")
        @classmethod
        def validate_stato_attivita(cls, v):
            print(v)
            try:
                pass
            except ValidationError as e:
                if isinstance(v, bool):
                    return v
                elif isinstance(v, str):
                    if v.lower() in ["attiva", "true", "1", "yes", "t"]:
                        return True
                    elif v.lower() in ["inattiva", "false", "0", "no", "f"]:
                        return False
                raise e
        
        @field_validator("data_inizio_attivita")
        @classmethod
        def validate_data_inizio_attivita(cls, v):
            return datetime.strptime(v, "%d/%m/%Y")

        
        @field_validator("ateco")
        @classmethod
        def validate_ateco(cls, v):
            return v.replace(".", "")
        
        @field_validator("nace")
        @classmethod
        def validate_nace(cls, v):
            return v.replace(".", "")
        
        @field_serializer("data_inizio_attivita")
        @classmethod
        def serialize_data_inizio_attivita(cls, v):
            return serialize_date(v)

class Addetti(BaseModel):
    anno: int = Field(title="Anno", description="Anno di a cui sono riferiti i dati sugli addetti riportati negli altri campi del modello")
    addetti_attuali: int = Field(title="Addetti attuali", description="Numero di addetti dell'anno di riferimento")
    addetti_attuali_dipendenti: int = Field(title="Addetti attuali dipendenti", description="Numero di addetti dipendenti dell'anno di riferimento")
    addetti_attuali_indipendenti: int = Field( title="Addetti attuali indipendenti", description="Numero di addetti indipendenti dell'anno di riferimento")

class DatiEconomici(BaseModel):
    cap_sociale_sottoscritto: float = Field( title="Capitale sociale sottoscritto", description="Capitale sociale sottoscritto dell'anno di riferimento")
    ultimo_valore_produzione: float = Field( title="Ultimo valore produzione", description="Ultimo valore produzione dell'anno di riferimento")
    ultimo_utile_precedente: float = Field( title="Ultimo utile precedente", description="Ultimo utile precedente dell'anno di riferimento")
    pratiche_ultimi_12_mesi: int = Field( title="Pratiche ultimi 12 mesi", description="Numero di pratiche depositate al registro delle imprese negli ultimi 12 mesi")
    n_trasferimenti_quote: int = Field( title="Trasferimenti quote", description="Numero di trasferimenti di quote nella vita della società")
    n_trasferimenti_sede: int = Field( title="Trasferimenti sede", description="Numero di trasferimenti di sede nella vita della società")

class LicenzeCertificazioni(BaseModel):
    tipologia_dati: str = Field(title="Tipologia dati", description="Tipologia dei dati riguardanti le licenze o certificazioni o altra autorizzazione es. Licenze, Certificazioni, Autorizzazioni, Attestazioni, etc.")
    tipo_licenza: str = Field(title="Tipo licenza", description="Tipo della licezza o certificazione es. SOA per la categoria OG2, ISO 9001, ISO 14001, etc.")
    estensore: str = Field(title="Estensore", description="Estensore della licenza, soggetto che ha rilasciato la licenza o certificazione")
    #data_emissione: datetime = Field(title="Data emissione", description="Data di emissione della licenza o certificazione")
    #data_scadenza: datetime = Field(title="Data scadenza", description="Data di scadenza della licenza o certificazione")
    data_emissione: str = Field(title="Data emissione", description="Data di emissione della licenza o certificazione")
    data_scadenza: str = Field(title="Data scadenza", description="Data di scadenza della licenza o certificazione")
    numero_licenza: str = Field(title="Numero licenza", description="Numero di licenza o certificazione")

    
class ValLicenzeCertificazioni(LicenzeCertificazioni):
    @field_validator("data_emissione")
    @classmethod
    def validate_data_emissione(cls, v):
        return datetime.strptime(v, "%d/%m/%Y")
    
    @field_validator("data_scadenza")
    @classmethod
    def validate_data_scadenza(cls, v):
        return datetime.strptime(v, "%d/%m/%Y")
    
    @field_serializer("data_emissione")
    @classmethod
    def serialize_data_emissione(cls, v):
        return serialize_date(v)
    
    @field_serializer("data_scadenza")
    @classmethod
    def serialize_data_scadenza(cls, v):
        return serialize_date(v)

class Sezione(BaseModel):
    tipo_sezione: str = Field(title="Tipo sezione", description="Tipologia della sezione es. Sezione artigiani, Sezione agricola, Sezione startup innovative, sezione Piccole e medie imprese (PMI) innovative, Incubatori certificati, Imprese sociali, Soggetti che esercitano attività di direzione e coordinamento e quelli a loro soggetti, società tra professionisti, etc.")
    numero_sezione: str = Field(title="Numero sezione", description="Numero identificativo della sezione")
    #data_istituzione: datetime = Field(title="Data istituzione", description="Data di iscrizione della sezione")
    data_istituzione: str = Field(title="Data istituzione", description="Data di iscrizione della sezione")

class ValSezione(Sezione):
    @field_validator("data_istituzione")
    @classmethod
    def validate_data_istituzione(cls, v):
        return datetime.strptime(v, "%d/%m/%Y")
    
    @field_serializer("data_istituzione")
    @classmethod
    def serialize_data_istituzione(cls, v):
        return serialize_date(v)

class Socio(BaseModel):
    ## sistema per caso sia persona fisica che giuridica
    #codice_fiscale: str = Field(pattern=r"[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]", title="Codice fiscale", description="Codice fiscale della persona fisica o giuridica")
    codice_fiscale: str = Field(title="Codice fiscale", description="Codice fiscale della persona fisica o giuridica")
    quota_sociale: float = Field( title="Quota sociale", description="Importo della auota sociale  detenuta dalla persona fisica o giuridica")
    titolo_di_diritto: str = Field(title="Titolo di diritto", description="Titolo di diritto per il quale è detenuta la quota sociale dalla persona fisica o giuridica es. proprietà, usufrutto, nuda proprietà, etc.")
    detentori_diritti: str = Field(title="Detentori diritti", description="Detentori dei diritti sulle quote sociali, diversi dal socio, es. usufruttuario, tutore, etc.")

class ValSocio(Socio):
    @field_validator("codice_fiscale")
    @classmethod
    def validate_codice_fiscale(cls, v):
        return re.match(r"[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]", v)

class ContrattoDiRete(BaseModel):
    numero_repertorio_contratto_di_rete: str = Field(title="Numero repertorio contratti di rete", description="Numero identificativo di repertorio notarile del contratto di rete")
    numero_registrazione_contratti_di_rete: str = Field(title="Numero registrazione contratti di rete", description="Numero identificativo del contratto di rete")
    denominazione_contratto_rete: str = Field(title="Denominazione contratto di rete", description="Denominazione del contratto di rete")

class PersonaFisica(BaseModel):
    nome: str = Field(title="Nome", description="Nome della persona fisica")
    cognome: str = Field(title="Cognome", description="Cognome della persona fisica")
    #data_nascita: datetime = Field(title="Data nascita", description="Data di nascita della persona fisica")
    data_nascita: str = Field(title="Data nascita", description="Data di nascita della persona fisica")
    luogo_nascita: str = Field(title="Luogo nascita", description="Luogo di nascita della persona fisica")
    ###! Regex sbagliato
    #codice_fiscale: str = Field(pattern=r"[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]", title="Codice fiscale", description="Codice fiscale della persona fisica")
    codice_fiscale: str = Field(title="Codice fiscale", description="Codice fiscale della persona fisica")

class ValPersonaFisica(PersonaFisica):
    @field_validator("codice_fiscale")
    @classmethod
    def validate_codice_fiscale(cls, v):
        return re.match(r"[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]", v)
    
    @field_validator("data_nascita")
    @classmethod
    def validate_data_nascita(cls, v):
        return datetime.strptime(v, "%d/%m/%Y")
    
    @field_serializer("data_nascita")
    @classmethod
    def serialize_data_nascita(cls, v):
        return serialize_date(v)

class Carica(PersonaFisica):
    #cf_amministratore: str = Field(pattern=r"[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]", title="Codice fiscale", description="Codice fiscale della persona fisica")
    cf_amministratore: str = Field(title="Codice fiscale", description="Codice fiscale della persona fisica")
    ruolo_amministratore: str = Field(title="Ruolo amministratore", description="Ruolo che la persona fisica ricopre come amministratore, revisiore o sindaco es. presidente del consiglio di amministratore, amminsitratore delegato, institore, procuratore, direttore generale, amministratore unico, vicepresidente, sindaco, sindaco supplente, presidente del consiglio sindacale, reviore, società di revisione, presidente del consiglio dei revisori, revisore unico, sindaco unico, etc.")
    #data_nomina_amministratore: datetime = Field(title="Data nomina amministratore", description="Data di nomina amministratore, sindaco o revisore della persona fisica")
    #data_cessazione_amministratore: datetime = Field(title="Data cessazione amministratore", description="Data di cessazione amministratore della persona fisica")
    data_nomina_amministratore: str = Field(title="Data nomina amministratore", description="Data di nomina amministratore, sindaco o revisore della persona fisica")
    data_cessazione_amministratore: str = Field(title="Data cessazione amministratore", description="Data di cessazione amministratore della persona fisica")

class ValCarica(Carica):
    @field_validator("cf_amministratore")
    @classmethod
    def validate_cf_amministratore(cls, v):
        return re.match(r"[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]", v)
    
    @field_validator("data_nomina_amministratore")
    @classmethod
    def validate_data_nomina_amministratore(cls, v):
        return datetime.strptime(v, "%d/%m/%Y") 
    
    @field_validator("data_cessazione_amministratore")
    @classmethod
    def validate_data_cessazione_amministratore(cls, v):
        return datetime.strptime(v, "%d/%m/%Y")
    
    @field_serializer("data_nomina_amministratore")
    @classmethod
    def serialize_data_nomina_amministratore(cls, v):
        return serialize_date(v)
    
    @field_serializer("data_cessazione_amministratore")
    @classmethod
    def serialize_data_cessazione_amministratore(cls, v):
        return serialize_date(v)

class SoggettoGiuridico(BaseModel):
    denominazione: str = Field(title="Denominazione", description="Denominazione del soggetto giuridico")
    #rea: str = Field(pattern=r"[A-Z]{2} - \d{6}", title="REA", description="Numero di repertorio economico amministrativo (REA) del soggetto giuridico")
    rea: str = Field(title="REA", description="Numero di repertorio economico amministrativo (REA) del soggetto giuridico")
    ## sistema per caso sia persona fisica che giuridica
    #codice_fiscale: str = Field(pattern=r"[A-Z]{6}\d{2}[A-Z]\d{2}[A-Z]\d{3}[A-Z]", title="Codice fiscale", description="Codice fiscale  del soggetto giuridico")
    #partita_iva: str = Field(pattern=r"\d{11}", title="Partita IVA", description="Partita IVA del soggetto giuridico")
    codice_fiscale: str = Field(title="Codice fiscale", description="Codice fiscale  del soggetto giuridico")
    partita_iva: str = Field(title="Partita IVA", description="Partita IVA del soggetto giuridico")
    forma_giuridica: str = Field(title="Forma giuridica", description="Forma giuridica del soggetto es. società a responsabilità limitata, società per azioni, società in nome collettivo, società in accomandita semplice, società in accomandita per azioni, società per azioni, consorzio, societtà cooperativa a responsabilità limitata, azienda speciale, associazione, fondazione, ente pubblico economico, ente pubblico etc.")
    pubblico: bool = Field(title="Pubblico", description="Indica se il soggetto è pubblico es. ente pubblico, società partecipatea pubblica")
    #termine_societario: datetime = Field(title="Termine societario", description="Data di termine del soggetto")
    #scadenza_esercizio_contabile: datetime = Field(title="Scadenza esercizio contabile", description="Data di scadenza dell'esercizio contabile del soggetto")
    termine_societario: str = Field(title="Termine societario", description="Data di termine del soggetto")
    scadenza_esercizio_contabile: str = Field(title="Scadenza esercizio contabile", description="Data di scadenza dell'esercizio contabile del soggetto")
    oggetto_sociale: str = Field(title="Oggetto sociale", description="Oggetto sociale del soggetto, scopo cui il soggetto è finalizzato")
    poteri_statutari: str = Field(title="Poteri statutari", description="Poteri statutari  deputati agli amministratori del soggetto")
    #data_istituzione: datetime = Field(title="Data istituzione", description="Data di istituzione del soggetto")
    #data_ultimo_protocollo: datetime = Field(title="Data ultimo protocollo", description="Data di ultimo protocollo al registro delle imprese del soggetto")
    data_istituzione: str = Field(title="Data istituzione", description="Data di istituzione del soggetto")
    data_ultimo_protocollo: str = Field(title="Data ultimo protocollo", description="Data di ultimo protocollo al registro delle imprese del soggetto")
    attivita: List[Attivita] = Field(title="Attività", description="Lista delle attività svolte dal soggetto")
    sedi: List[Sede] = Field(title="Sedi", description="Lista delle sedi del soggetto")
    addetti: List[Addetti] = Field(title="Addetti", description="degli addetti del soggetto")
    dati_economici: List[DatiEconomici] = Field(title="Dati economici", description="Lista dei dati economici del soggetto")
    licenze_certificazioni: List[LicenzeCertificazioni] = Field(title="Licenze certificazioni", description="Lista delle licenze e certificazioni del soggetto")
    #data_iscrizione: datetime = Field(title="Data iscrizione", description="Data di iscrizione del soggetto")
    #data_atto_costitutivo: datetime = Field(title="Data atto costitutivo", description="Data di costituzione del soggetto")
    data_iscrizione: str = Field(title="Data iscrizione", description="Data di iscrizione del soggetto")
    data_atto_costitutivo: str = Field(title="Data atto costitutivo", description="Data di costituzione del soggetto")
    sistema_amministrativo: str = Field(title="Sistema amministrativo", description="Sistema amministrativo del soggetto ad es. consiglio di amministrazione, amministratore unico, sistema dualistico, sistema monoisico etc.")
    precedente_numero_iscrizione: str = Field(title="Precedente numero iscrizione", description="Numero di iscrizione precedente del soggetto")
    sezioni: List[Sezione] = Field(title="Sezioni", description="Lista delle sezioni in cui è iscritto il soggetto")
    cap_sociale_sottoscritto: float = Field( title="Capitale sociale sottoscritto", description="Capitale sociale sottoscritto del soggetto")
    cap_sociale_deliberato: float = Field( title="Capitale sociale deliberato", description="Capitale sociale deliberato del soggetto" )
    cap_sociale_versato: float = Field( title="Capitale sociale versato", description="Capitale sociale versato del soggetto"  )
    soci: List[Socio] = Field(title="Soci", description="Lista dei soci del soggetto")
    contratti_di_rete: List[ContrattoDiRete] = Field(title="Contratti di rete", description="Lista dei contratti di rete in cui il soggetto è parte")
    amministratori: List[Carica] = Field(title="Amministratori", description="Lista degli amministratori del soggetto")
    sindaci_membri_organi_di_controllo: List[Carica] = Field(title="Sindaci membri organi di controllo", description="Lista dei sindaci, revisori e membri degli organi di controllo del soggetto")

class ValSoggettoGiuridico(SoggettoGiuridico):
    @field_validator("data_istituzione")
    @classmethod
    def validate_data_istituzione(cls, v):
        return datetime.strptime(v, "%d/%m/%Y")
    
    @field_validator("data_ultimo_protocollo")
    @classmethod
    def validate_data_ultimo_protocollo(cls, v):
        return datetime.strptime(v, "%d/%m/%Y") 
    
    @field_validator("data_iscrizione")
    @classmethod
    def validate_data_iscrizione(cls, v):
        return datetime.strptime(v, "%d/%m/%Y")
    
    @field_validator("data_atto_costitutivo")
    @classmethod
    def validate_data_atto_costitutivo(cls, v): 
        return datetime.strptime(v, "%d/%m/%Y")
    
    @field_validator("termine_societario")
    @classmethod
    def validate_termine_societario(cls, v):
        return datetime.strptime(v, "%d/%m/%Y")

    @field_validator("scadenza_esercizio_contabile")
    @classmethod
    def validate_scadenza_esercizio_contabile(cls, v):
        return datetime.strptime(v, "%d/%m")
    
    @field_validator("attivita")
    @classmethod
    def validate_attivita(cls, attivitas):
        return [ValAttivita.model_validate_json(attivita.model_dump_json()) for attivita in attivitas]
    
    @field_serializer("data_istituzione")
    @classmethod
    def serialize_data_istituzione(cls, v):
        return serialize_date(v)
    
    @field_serializer("data_ultimo_protocollo")
    @classmethod
    def serialize_data_ultimo_protocollo(cls, v):
        return serialize_date(v)
    
    @field_serializer("data_iscrizione")
    @classmethod
    def serialize_data_iscrizione(cls, v):
        return serialize_date(v)
    
    @field_serializer("data_atto_costitutivo")
    @classmethod
    def serialize_data_atto_costitutivo(cls, v):
        return serialize_date(v)
    
    @field_serializer("termine_societario")
    @classmethod
    def serialize_termine_societario(cls, v):
        return serialize_date(v)
    
    @field_serializer("scadenza_esercizio_contabile")
    @classmethod
    def serialize_scadenza_esercizio_contabile(cls, v):
        return serialize_date(v, format="%d/%m")

In [92]:
response = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Estrai le informazioni dal testo e restituisci un oggetto JSON"},
        {"role": "user", "content": text},
    ], response_format=SoggettoGiuridico
)


In [93]:
response.choices[0].message.content

'{\n  "denominazione": "VICENZA E\' - SOCIETA\' CONSORTILE A RESPONSABILITA\' LIMITATA",\n  "rea": "VI - 261351",\n  "codice_fiscale": "02615450240",\n  "partita_iva": "02615450240",\n  "forma_giuridica": "SOCIETA\' CONSORTILE A RESPONSABILITA\' LIMITATA",\n  "pubblico": false,\n  "termine_societario": "31/12/2030",\n  "scadenza_esercizio_contabile": "31/12",\n  "oggetto_sociale": "LA SOCIETA\' HA PER OGGETTO L\'ISTITUZIONE DI UNA ORGANIZZAZIONE COMUNE FRA I SOCI INTERESSATI AL SETTORE DI FILIERA DEL TURISMO, PER LO SVOLGIMENTO E LA REGOLAMENTAZIONE DI DETERMINATE FASI DELLE ATTIVITA\' D\'IMPRESA DEI SOCI STESSI.",\n  "poteri_statutari": "LA GESTIONE DELLA SOCIETA\' E\' ATTRIBUITA ALL\'ORGANO AMMINISTRATIVO; A TAL FINE TALE ORGANO PUO\' COMPIERE TUTTI GLI ATTI E LE OPERAZIONI SIA DI ORDINARIA CHE DI STRAORDINARIA AMMINISTRAZIONE.",\n  "data_istituzione": "13/05/1997",\n  "data_ultimo_protocollo": "12/06/2024",\n  "attivita": [\n    {\n      "stato_attivita": true,\n      "prevalente_bo

In [121]:
Impresa = ValSoggettoGiuridico.model_validate_json(response.choices[0].message.content)

True


In [122]:
Impresa.model_dump_json()

/Users/amosbrazzoli/.local/share/virtualenvs/Visure-_1oFNKyX/lib/python3.11/site-packages/pydantic/main.py:477: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `datetime` with value `datetime.datetime(1997, 8, 7, 0, 0)` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_json(


'{"denominazione":"VICENZA E\' - SOCIETA\' CONSORTILE A RESPONSABILITA\' LIMITATA","rea":"VI - 261351","codice_fiscale":"02615450240","partita_iva":"02615450240","forma_giuridica":"SOCIETA\' CONSORTILE A RESPONSABILITA\' LIMITATA","pubblico":false,"termine_societario":"31/12/2030","scadenza_esercizio_contabile":"31/12","oggetto_sociale":"LA SOCIETA\' HA PER OGGETTO L\'ISTITUZIONE DI UNA ORGANIZZAZIONE COMUNE FRA I SOCI INTERESSATI AL SETTORE DI FILIERA DEL TURISMO, PER LO SVOLGIMENTO E LA REGOLAMENTAZIONE DI DETERMINATE FASI DELLE ATTIVITA\' D\'IMPRESA DEI SOCI STESSI.","poteri_statutari":"LA GESTIONE DELLA SOCIETA\' E\' ATTRIBUITA ALL\'ORGANO AMMINISTRATIVO; A TAL FINE TALE ORGANO PUO\' COMPIERE TUTTI GLI ATTI E LE OPERAZIONI SIA DI ORDINARIA CHE DI STRAORDINARIA AMMINISTRAZIONE.","data_istituzione":"13/05/1997","data_ultimo_protocollo":"12/06/2024","attivita":[{"stato_attivita":null,"prevalente_boolean":true,"data_inizio_attivita":"1997-08-07T00:00:00","descrizione_attivita_prevalent

In [ ]:
'{"denominazione":"VICENZA E\' - SOCIETA\' CONSORTILE A RESPONSABILITA\' LIMITATA","rea":"VI - 261351","codice_fiscale":"02615450240","partita_iva":"02615450240","forma_giuridica":"SOCIETA\' CONSORTILE A RESPONSABILITA\' LIMITATA","pubblico":false,"termine_societario":"31/12/2030","scadenza_esercizio_contabile":"31/12","oggetto_sociale":"LA SOCIETA\' HA PER OGGETTO L\'ISTITUZIONE DI UNA ORGANIZZAZIONE COMUNE FRA I SOCI INTERESSATI AL SETTORE DI FILIERA DEL TURISMO, PER LO SVOLGIMENTO E LA REGOLAMENTAZIONE DI DETERMINATE FASI DELLE ATTIVITA\' D\'IMPRESA DEI SOCI STESSI.","poteri_statutari":"LA GESTIONE DELLA SOCIETA\' E\' ATTRIBUITA ALL\'ORGANO AMMINISTRATIVO; A TAL FINE TALE ORGANO PUO\' COMPIERE TUTTI GLI ATTI E LE OPERAZIONI SIA DI ORDINARIA CHE DI STRAORDINARIA AMMINISTRAZIONE.","data_istituzione":"13/05/1997","data_ultimo_protocollo":"12/06/2024","attivita":[{"stato_attivita":null,"prevalente_boolean":true,"data_inizio_attivita":"1997-08-07T00:00:00","descrizione_attivita_prevalente":"SERVIZI DI ASSISTENZA AI VISITATORI: FORNITURA DI INFORMAZIONI TURISTICHE AI VIAGGIATORI.","ateco":"799019","nace":"799","import_export_boolean":false}],"sedi":[{"tipo_via":"VIA","via":"MONTALE 25","cap":"36100","comune":"VICENZA","provincia":"VI","regione":"VENETO","paese":"ITALIA","tipo_sede":"Sede legale"}],"addetti":[{"anno":2015,"addetti_attuali":1,"addetti_attuali_dipendenti":1,"addetti_attuali_indipendenti":0}],"dati_economici":[{"cap_sociale_sottoscritto":14248.0,"ultimo_valore_produzione":41479.0,"ultimo_utile_precedente":394.0,"pratiche_ultimi_12_mesi":4,"n_trasferimenti_quote":37,"n_trasferimenti_sede":0}],"licenze_certificazioni":[],"data_iscrizione":"07/08/1997","data_atto_costitutivo":"13/05/1997","sistema_amministrativo":"CONSIGLIO DI AMMINISTRAZIONE","precedente_numero_iscrizione":"VI-1997-185992","sezioni":[{"tipo_sezione":"ORDINARIA","numero_sezione":"ORD-001","data_istituzione":"07/08/1997"}],"cap_sociale_sottoscritto":14248.0,"cap_sociale_deliberato":14248.0,"cap_sociale_versato":14248.0,"soci":[{"codice_fiscale":"02222540243","quota_sociale":1768.0,"titolo_di_diritto":"PROPRIETA","detentori_diritti":"Nessuno"},{"codice_fiscale":"05501530967","quota_sociale":520.0,"titolo_di_diritto":"PROPRIETA","detentori_diritti":"Nessuno"},{"codice_fiscale":"04122700240","quota_sociale":416.0,"titolo_di_diritto":"PROPRIETA","detentori_diritti":"Nessuno"},{"codice_fiscale":"00543220248","quota_sociale":312.0,"titolo_di_diritto":"PROPRIETA","detentori_diritti":"Nessuno"},{"codice_fiscale":"DLLLNE38T61D717A","quota_sociale":260.0,"titolo_di_diritto":"PROPRIETA","detentori_diritti":"Nessuno"},{"codice_fiscale":"00178740247","quota_sociale":260.0,"titolo_di_diritto":"PROPRIETA","detentori_diritti":"Nessuno"},{"codice_fiscale":"01276640248","quota_sociale":260.0,"titolo_di_diritto":"PROPRIETA","detentori_diritti":"Nessuno"},{"codice_fiscale":"03328860246","quota_sociale":208.0,"titolo_di_diritto":"PROPRIETA","detentori_diritti":"Nessuno"},{"codice_fiscale":"03658500248","quota_sociale":208.0,"titolo_di_diritto":"PROPRIETA","detentori_diritti":"Nessuno"}],"contratti_di_rete":[{"numero_repertorio_contratto_di_rete":"VI-261351","numero_registrazione_contratti_di_rete":"534","denominazione_contratto_rete":"TVB TERRE VICENTINE BIKE"}],"amministratori":[{"nome":"STEFANIA","cognome":"PARISE","data_nascita":"11/11/1973","luogo_nascita":"MAROSTICA (VI)","codice_fiscale":"PRSSFN73S51E970Q","cf_amministratore":"PRSSFN73S51E970Q","ruolo_amministratore":"CONSIGLIERA","data_nomina_amministratore":"15/05/2024","data_cessazione_amministratore":"N/A"},{"nome":"PAOLO","cognome":"ASNICAR","data_nascita":"29/06/1963","luogo_nascita":"RECOARO TERME (VI)","codice_fiscale":"SNCPLA63H29H214Z","cf_amministratore":"SNCPLA63H29H214Z","ruolo_amministratore":"CONSIGLIERE","data_nomina_amministratore":"15/05/2024","data_cessazione_amministratore":"N/A"},{"nome":"OSCAR","cognome":"ZAGO","data_nascita":"24/02/1969","luogo_nascita":"VERONA (VR)","codice_fiscale":"ZGASCR69B24L781L","cf_amministratore":"ZGASCR69B24L781L","ruolo_amministratore":"CONSIGLIERE","data_nomina_amministratore":"15/05/2024","data_cessazione_amministratore":"N/A"},{"nome":"ANTONIO","cognome":"CHEMELLO","data_nascita":"08/06/1964","luogo_nascita":"SANDRIGO (VI)","codice_fiscale":"CHMNTN64H08H829S","cf_amministratore":"CHMNTN64H08H829S","ruolo_amministratore":"CONSIGLIERE","data_nomina_amministratore":"15/05/2024","data_cessazione_amministratore":"N/A"},{"nome":"PAOLO","cognome":"COGO","data_nascita":"06/08/1976","luogo_nascita":"VICENZA (VI)","codice_fiscale":"CGOPLA76M06L840C","cf_amministratore":"CGOPLA76M06L840C","ruolo_amministratore":"CONSIGLIERE","data_nomina_amministratore":"15/05/2024","data_cessazione_amministratore":"N/A"},{"nome":"CATERINA","cognome":"BONOTTO","data_nascita":"28/07/1966","luogo_nascita":"MAROSTICA (VI)","codice_fiscale":"BNTCRN66L68E970D","cf_amministratore":"BNTCRN66L68E970D","ruolo_amministratore":"CONSIGLIERA","data_nomina_amministratore":"15/05/2024","data_cessazione_amministratore":"N/A"},{"nome":"CARLO ALBERTO","cognome":"CANDIA","data_nascita":"14/04/1984","luogo_nascita":"ARZIGNANO (VI)","codice_fiscale":"CNDCLL84D14A459N","cf_amministratore":"CNDCLL84D14A459N","ruolo_amministratore":"CONSIGLIERE","data_nomina_amministratore":"15/05/2024","data_cessazione_amministratore":"N/A"},{"nome":"DENISE","cognome":"LAPPO","data_nascita":"10/06/1977","luogo_nascita":"MONTECCHIO MAGGIORE (VI)","codice_fiscale":"LPPDNS77H50F464J","cf_amministratore":"LPPDNS77H50F464J","ruolo_amministratore":"CONSIGLIERA","data_nomina_amministratore":"15/05/2024","data_cessazione_amministratore":"N/A"},{"nome":"GIOVANNI","cognome":"PAVESI","data_nascita":"15/10/1965","luogo_nascita":"MANTOVA (MN)","codice_fiscale":"PVSGNN65R15E897K","cf_amministratore":"PVSGNN65R15E897K","ruolo_amministratore":"CONSIGLIERE","data_nomina_amministratore":"15/05/2024","data_cessazione_amministratore":"N/A"},{"nome":"ALDO","cognome":"ZANCONATO","data_nascita":"05/03/1992","luogo_nascita":"ARZIGNANO (VI)","codice_fiscale":"ZNCFNC92C05A459O","cf_amministratore":"ZNCFNC92C05A459O","ruolo_amministratore":"CONSIGLIERE","data_nomina_amministratore":"15/05/2024","data_cessazione_amministratore":"N/A"}],"sindaci_membri_organi_di_controllo":[]}'
